In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_6768/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,9

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [ ]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

In [8]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb


In [9]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'xgb']]}
)

In [10]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[15:02:40] Stdout logging level is INFO2.
[15:02:40] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[15:02:40] Task: multilabel

[15:02:40] Start automl preset with listed constraints:
[15:02:40] - time: 3600.00 seconds
[15:02:40] - CPU: 1 cores
[15:02:40] - memory: 16 GB

[15:02:40] Train data shape: (14644, 22)
[15:02:41] Feats was rejected during automatic roles guess: []
[15:02:41] Layer 1 train process start. Time left 3599.34 secs
[15:02:41] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:02:42] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[15:02:48] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[15:02:53] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[15:03:00] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.3779354299948015
[15:03:00] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[15:03:00] Time left 3580.54 

In [11]:
gpu_inf = automl_gpu.predict(data)

In [12]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[15:04:38] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[15:04:38] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[15:04:38] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[15:04:38] CatBoost uses as obj. MultiCrossEntropy.


In [13]:
cpu_inf = automl_gpu.predict(data)

In [14]:
from joblib import dump, load
import time
pickle_file = './gpu.joblib'

In [15]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_gpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.398s


In [31]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 0.289s


In [32]:
disk_pred = model_inf.predict(data)

In [33]:
disk_pred.data.T

array([[ 1.2499781 ,  1.2954457 ,  1.215799  , ...,  1.7942188 ,
         1.1554844 ,  1.4735762 ],
       [-1.3430822 , -1.4893754 , -1.3048111 , ..., -2.0758078 ,
        -1.1770225 , -1.7535352 ],
       [-1.9479339 , -1.6672715 , -1.8437766 , ..., -2.1467757 ,
        -1.7655997 , -2.5173562 ],
       ...,
       [ 0.57334274,  0.63242227,  0.54720515, ...,  1.3476214 ,
         0.46615654,  0.8401307 ],
       [-1.1950628 , -1.0716614 , -1.2032543 , ..., -1.1580757 ,
        -1.2645098 , -1.0374529 ],
       [ 0.0785979 ,  0.3153556 ,  0.12578224, ...,  1.3680722 ,
         0.11524606, -0.06934854]], dtype=float32)

In [19]:
disk_pred.data.T

array([[ 1.301159  ,  1.34741   ,  1.2640569 , ...,  1.8646438 ,
         1.2030007 ,  1.5263661 ],
       [-1.2656877 , -1.416658  , -1.2259802 , ..., -2.0270844 ,
        -1.0922014 , -1.6940601 ],
       [-1.9110572 , -1.6158959 , -1.8038567 , ..., -2.149033  ,
        -1.7254399 , -2.494409  ],
       ...,
       [ 0.6196411 ,  0.6809143 ,  0.59041625, ...,  1.421854  ,
         0.5145211 ,  0.8948479 ],
       [-1.166754  , -1.0429802 , -1.1768596 , ..., -1.131111  ,
        -1.2407193 , -0.99694276],
       [ 0.128881  ,  0.3690266 ,  0.17519997, ...,  1.44401   ,
         0.16227192, -0.0143208 ]], dtype=float32)

In [20]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
#print()
#print(oof_pred.data.T)
#print()
#print(disk_pred.data.T)

[[ 1.301159    1.34741     1.2640569  ...  1.8646438   1.2030007
   1.5263661 ]
 [-1.2656877  -1.416658   -1.2259802  ... -2.0270844  -1.0922014
  -1.6940601 ]
 [-1.9110572  -1.6158959  -1.8038567  ... -2.149033   -1.7254399
  -2.494409  ]
 ...
 [ 0.6196411   0.6809143   0.59041625 ...  1.421854    0.5145211
   0.8948479 ]
 [-1.166754   -1.0429802  -1.1768596  ... -1.131111   -1.2407193
  -0.99694276]
 [ 0.128881    0.3690266   0.17519997 ...  1.44401     0.16227192
  -0.0143208 ]]

[[0.10802634 0.13113078 0.09910143 ... 0.21198082 0.09147822 0.12580821]
 [0.0066313  0.00730639 0.00573334 ... 0.0023582  0.00835318 0.00566869]
 [0.00248677 0.0051474  0.00280998 ... 0.00225817 0.00328291 0.00223576]
 ...
 [0.04944737 0.06560731 0.0530625  ... 0.14098077 0.04365823 0.0626318 ]
 [0.00599362 0.01186266 0.0063459  ... 0.01556174 0.00508943 0.00692001]
 [0.02530477 0.03941411 0.02897311 ... 0.13118058 0.02947844 0.02688901]]

[[0.10372297 0.13033417 0.10176552 ... 0.22322935 0.08946411 0.1356

In [21]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 6768. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-12-29 15:09:31,146 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-29 15:09:31,146 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:49767': None}

In [22]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[15:09:31] CatBoost uses as obj. MultiCrossEntropy.
[15:09:31] Stdout logging level is INFO2.
[15:09:31] Task: multilabel

[15:09:31] Start automl preset with listed constraints:
[15:09:31] - time: 3600.00 seconds
[15:09:31] - CPU: 1 cores
[15:09:31] - memory: 16 GB

[15:09:31] Train data shape: (14644, 22)
[15:09:32] Feats was rejected during automatic roles guess: []
[15:09:32] Layer 1 train process start. Time left 3599.39 secs
[15:09:32] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:09:32] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[15:09:37] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[15:09:41] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[15:09:49] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.376846383376888
[15:09:49] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[15:09:49] Time left 3582.22 secs

[15:09:49] Stdout l

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:09:55] task [xgboost.dask-0]:ucx://127.0.0.1:49767 got new rank 0


[15:10:29] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:10:29] task [xgboost.dask-0]:ucx://127.0.0.1:49767 got new rank 0


[15:11:01] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:11:02] task [xgboost.dask-0]:ucx://127.0.0.1:49767 got new rank 0


[15:11:34] Fitting Lvl_0_Pipe_1_Mod_0_XGB finished. score = -7.421265013672653
[15:11:34] Lvl_0_Pipe_1_Mod_0_XGB fitting and predicting completed
[15:11:34] Time left 3477.58 secs

[15:11:34] Layer 1 training completed.

[15:11:34] Blending: Optimization starts with equal weights and score -7.372156143351286
[15:11:34] Blending, iter 0: score = -7.333655279769789, weights = [0.90983003 0.09016994]
[15:11:34] Blending, iter 1: score = -7.333655279769789, weights = [0.90983003 0.09016994]
[15:11:34] No score update. Terminated
[15:11:34] Automl preset training completed in 122.60 seconds

[15:11:34] Model description:
Final prediction for new objects (level 0) = 
	 0.90983 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.09017 * (3 averaged models Lvl_0_Pipe_1_Mod_0_XGB) 



In [23]:
automl_mgpu.to_cpu()

multilabel isn`t supported in lgb
[15:11:34] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[15:11:34] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[15:11:34] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[15:11:34] CatBoost uses as obj. MultiCrossEntropy.


In [24]:
mcpu_inf = automl_mgpu.predict(data)

In [26]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
#print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

[[ 1.301159    1.34741     1.2640569  ...  1.8646438   1.2030007
   1.5263661 ]
 [-1.2656877  -1.416658   -1.2259802  ... -2.0270844  -1.0922014
  -1.6940601 ]
 [-1.9110572  -1.6158959  -1.8038567  ... -2.149033   -1.7254399
  -2.494409  ]
 ...
 [ 0.6196411   0.6809143   0.59041625 ...  1.421854    0.5145211
   0.8948479 ]
 [-1.166754   -1.0429802  -1.1768596  ... -1.131111   -1.2407193
  -0.99694276]
 [ 0.128881    0.3690266   0.17519997 ...  1.44401     0.16227192
  -0.0143208 ]]

[[0.10802634 0.13113078 0.09910143 ... 0.21198082 0.09147822 0.12580821]
 [0.0066313  0.00730639 0.00573334 ... 0.0023582  0.00835318 0.00566869]
 [0.00248677 0.0051474  0.00280998 ... 0.00225817 0.00328291 0.00223576]
 ...
 [0.04944737 0.06560731 0.0530625  ... 0.14098077 0.04365823 0.0626318 ]
 [0.00599362 0.01186266 0.0063459  ... 0.01556174 0.00508943 0.00692001]
 [0.02530477 0.03941411 0.02897311 ... 0.13118058 0.02947844 0.02688901]]

[[0.10372297 0.13033417 0.10176552 ... 0.22322935 0.08946411 0.1356

In [30]:
from joblib import dump, load
import time

pickle_file = './mgpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.374s
